In [1]:
from functions import *
import torch

    
net = Net(3)
T = torch.tensor([1])
dt = torch.tensor([0.01])
S0 = torch.tensor([100])
q = torch.tensor([0.01])
A = torch.tensor([3, 4, 5])
B = torch.tensor([0.01, 0.01, 0.01])
Q = torch.tensor([100])
z = torch.tensor([1, 2, 3])
delta = torch.tensor([0.01])
gamma = torch.tensor([0.01])
sigma = torch.tensor([0.01])

# use cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
T = T.to(device)
dt = dt.to(device)
S0 = S0.to(device)
q = q.to(device)
A = A.to(device)
B = B.to(device)
Q = Q.to(device)
z = z.to(device)
delta = delta.to(device)
gamma = gamma.to(device)
sigma = sigma.to(device)

In [2]:
new_Net = ResNet(3, num_classes=1)
new_Net.to(device)

ResNet(
  (conv1): Conv1d(3, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inp

In [3]:
new_Net(T, S0, q)

RuntimeError: Given groups=1, weight of size [64, 3, 7], expected input[1, 1, 3] to have 3 channels, but got 1 channels instead

In [2]:
S, buy_orders, sell_orders, q, t = Train_Data_Simulation(1, 0.01, 0.1, S0, A, B, Q, z, delta, gamma, net)
total_loss(net, S, q, t, 0.001, buy_orders, sell_orders, z, delta, Q, A, B, gamma)

tensor(644.5459, device='cuda:0', grad_fn=<MulBackward0>)

In [3]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epoch in range(200):
    S, buy_orders, sell_orders, q, t = Train_Data_Simulation(1, 0.01, 0.1, S0, A, B, Q, z, delta, gamma, net)
    loss = total_loss(net, S, q, t, 0.001, buy_orders, sell_orders, z, delta, Q, A, B, gamma)
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

573.9168701171875
758.518310546875
572.6505737304688
550.671875


KeyboardInterrupt: 